# 🔗 OpenAI Embeddings - 텍스트를 숫자로 변환하는 마법

## 📚 개요

**문서 임베딩(Document Embedding)**은 텍스트를 컴퓨터가 이해할 수 있는 **수치형 벡터**로 변환하는 핵심 기술입니다! 🎯

### 🤔 임베딩이 뭐길래 이렇게 중요할까?

일상생활로 비유해보면, 임베딩은 **번역기**와 같습니다:

- **📝 텍스트** = 한국어 문장
- **🔢 벡터** = 영어로 번역된 문장  
- **🧠 AI 모델** = 번역 전문가

컴퓨터는 숫자만 이해할 수 있기 때문에, 문자를 숫자로 변환해야 AI가 텍스트의 의미를 파악할 수 있습니다!

### 🎯 OpenAI Embeddings의 특별한 점

- **🎨 의미 이해**: 단순한 단어 매칭이 아닌 **문맥적 의미** 파악
- **📊 고성능**: GPT 모델 기반의 뛰어난 벡터 생성
- **⚡ 효율성**: 다양한 크기 옵션으로 용도에 맞는 선택 가능

### 🔍 이 튜토리얼에서 배울 내용

1. **⚙️ 환경 설정** - OpenAI API 준비하기
2. **🔧 모델 정보** - 어떤 모델을 언제 써야 할까?
3. **🔍 쿼리 임베딩** - 단일 텍스트를 벡터로 변환
4. **📚 문서 임베딩** - 여러 문서를 한 번에 처리
5. **📏 차원 조정** - 벡터 크기 최적화하기
6. **🎯 유사도 계산** - 텍스트 간 유사성 측정하기

### 💡 실생활 활용 예시

- **🔍 검색 엔진**: "맛있는 파스타 레시피" → 관련 문서 찾기
- **🤖 챗봇**: 사용자 질문의 의도 파악
- **📊 문서 분류**: 이메일 스팸 감지, 뉴스 카테고리 분류
- **💝 추천 시스템**: 비슷한 상품이나 콘텐츠 추천

[📖 OpenAI 임베딩 공식 문서 더 알아보기](https://platform.openai.com/docs/guides/embeddings/embedding-models)

---

# 1. 환경 설정 🛠️

OpenAI Embeddings를 사용하기 위한 **필수 준비 작업**을 진행해봅시다! API 키 설정과 추적 기능 활성화가 필요합니다.

In [ ]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv(override=True)

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("LangChain-Tutorial")

# 2. 모델 정보 📊

OpenAI에서 제공하는 **3가지 임베딩 모델**을 비교해보세요! 각 모델마다 **성능**, **비용**, **용도**가 다릅니다.

## 📋 모델 비교표

| 모델명                   | 페이지당 비용  | MTEB 성능 점수 | 최대 입력 길이 | 🎯 추천 용도 |
|------------------------|-------------|-------------|------------|------------|
| **text-embedding-3-small** | 62,500      | 62.3%       | 8,191      | ⭐ **일반적 용도** (가성비 최고) |
| **text-embedding-3-large** | 9,615       | 64.6%       | 8,191      | 🎯 **고정밀 작업** (최고 성능) |
| **text-embedding-ada-002** | 12,500      | 61.0%       | 8,191      | 📜 **레거시 지원** (구버전) |

## 💡 어떤 모델을 선택해야 할까?

### 🥇 **text-embedding-3-small** (이 튜토리얼의 선택!)
- **장점**: 뛰어난 가성비, 빠른 속도
- **추천 상황**: 일반적인 문서 검색, 챗봇, 추천 시스템
- **💰 비용**: 가장 경제적

### 🏆 **text-embedding-3-large** 
- **장점**: 최고 성능, 미세한 의미 차이도 포착
- **추천 상황**: 법률 문서, 의료 텍스트 등 정밀도가 중요한 경우
- **💰 비용**: 가장 비쌈

### 📚 **text-embedding-ada-002**
- **용도**: 기존 시스템 호환성 유지
- **⚠️ 주의**: 신규 프로젝트에서는 3세대 모델 사용 권장

In [ ]:
from langchain_openai import OpenAIEmbeddings

# OpenAI의 "text-embedding-3-small" 모델을 사용하여 임베딩 객체를 생성합니다.
# 가성비가 좋고 일반적인 용도에 적합한 모델입니다.
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [ ]:
# 임베딩 테스트를 위한 한국어 샘플 문장을 정의합니다.
# 이 문장이 숫자 벡터로 변환될 예정입니다.
text = "임베딩 테스트를 하기 위한 샘플 문장입니다."

# 3. 쿼리 임베딩 🔍

**쿼리 임베딩**은 **하나의 텍스트**를 벡터로 변환하는 가장 기본적인 방법입니다!

## 🎯 embed_query() 메소드란?

`embeddings.embed_query(text)` 함수는 **단일 텍스트를 1536차원의 벡터로 변환**하는 핵심 함수입니다!

### 📚 어떻게 작동하나요?

1. **📝 입력**: 텍스트 문장 (예: "안녕하세요")
2. **🔄 처리**: OpenAI API를 통해 의미 분석
3. **📊 출력**: 1536개 숫자로 구성된 리스트 (벡터)

### 💡 주요 활용 사례

- **🔍 검색**: 사용자 질문과 가장 유사한 문서 찾기
- **📊 분석**: 텍스트의 감정, 주제 등 분석
- **🤖 챗봇**: 사용자 의도 파악하기

In [ ]:
# 텍스트를 OpenAI API를 통해 임베딩 벡터로 변환합니다.
# 결과는 1536개의 실수로 구성된 리스트가 됩니다.
query_result = embeddings.embed_query(text)

## 📊 결과 확인하기

생성된 임베딩 벡터의 **처음 5개 값**을 확인해봅시다! 

**실제 벡터는 1536개의 숫자**로 구성되어 있지만, 너무 길어서 앞부분만 미리보기로 확인해보겠습니다. 🔍

In [ ]:
# 임베딩 벡터의 처음 5개 요소를 슬라이싱으로 확인합니다.
# 실제로는 1536개의 float 값이 들어있습니다.
query_result[:5]

# 4. 문서 임베딩 📚

**문서 임베딩**은 **여러 개의 텍스트를 한 번에** 벡터로 변환하는 효율적인 방법입니다!

## 🎯 embed_documents() 메소드란?

`embeddings.embed_documents()` 함수는 **여러 문서를 일괄 처리**하는 강력한 기능입니다!

### 📚 쿼리 vs 문서 임베딩의 차이점

| 구분 | 🔍 쿼리 임베딩 | 📚 문서 임베딩 |
|------|-------------|-------------|
| **입력** | 하나의 텍스트 | 텍스트 리스트 |
| **출력** | 단일 벡터 | 벡터 리스트 |
| **용도** | 검색 쿼리, 사용자 질문 | 문서 컬렉션, 데이터베이스 구축 |

### 🚀 배치 처리의 장점

- **⚡ 속도**: 한 번에 여러 문서 처리로 시간 절약
- **💰 비용**: API 호출 횟수 감소로 비용 절약  
- **🛡️ 안정성**: 네트워크 오류 위험 감소

In [ ]:
# 동일한 텍스트를 4개 포함한 리스트를 일괄 임베딩 처리합니다.
# 결과는 4개의 벡터로 구성된 리스트가 됩니다.
doc_result = embeddings.embed_documents(
    [text, text, text, text]
)  # 텍스트를 임베딩하여 문서 벡터를 생성합니다.

## 📊 결과 분석하기

문서 임베딩의 **구조**를 확인해봅시다! 

- `doc_result`는 **4개의 벡터를 담은 리스트**입니다
- 각 벡터는 **1536개의 숫자**로 구성됩니다

In [ ]:
# 문서 임베딩 결과의 개수를 확인합니다 (4개 문서 = 4개 벡터)
len(doc_result)  # 문서 벡터의 길이를 확인합니다.

In [ ]:
# 첫 번째 문서의 임베딩 벡터에서 처음 5개 값을 확인합니다.
# [0]은 첫 번째 문서, [:5]는 처음 5개 요소를 의미합니다.
doc_result[0][:5]

# 5. 차원 조정 📏

`text-embedding-3` 시리즈 모델의 **특별한 기능** 중 하나는 **벡터 차원을 자유롭게 조정**할 수 있다는 것입니다!

## 🎯 차원 조정이 왜 필요할까?

### 🏠 아파트 비유로 이해하기

임베딩 벡터를 **아파트**에 비유해보세요:

- **🏢 1536차원**: 초고층 아파트 (정보 많음, 저장공간 많이 필요)
- **🏘️ 1024차원**: 중층 아파트 (적당한 정보, 적당한 공간)
- **🏘️ 512차원**: 저층 아파트 (기본 정보, 공간 절약)

### 💡 언제 차원을 줄일까?

- **💰 비용 절약**: 저장 공간과 계산 시간 감소
- **⚡ 속도 향상**: 벡터 연산이 더 빨라짐
- **🔋 메모리 절약**: 제한된 환경에서 효율적 운용

기본적으로 **text-embedding-3-small**은 **1536차원**의 벡터를 생성합니다.

In [ ]:
# 첫 번째 문서 벡터의 차원 수를 확인합니다.
# text-embedding-3-small 모델의 기본 차원은 1536입니다.
len(doc_result[0])

## ⚙️ 차원(dimensions) 조정하기

`dimensions` 매개변수를 사용하면 벡터 크기를 자유롭게 조정할 수 있습니다! 

### 📊 성능 vs 효율성 트레이드오프

| 차원 수 | 🎯 정확도 | ⚡ 속도 | 💾 저장공간 | 🎯 추천 용도 |
|--------|----------|-------|-----------|------------|
| **1536** | 최고 | 보통 | 많음 | 고정밀 작업 |
| **1024** | 높음 | 빠름 | 보통 | **일반적 용도** ⭐ |
| **512** | 보통 | 매우 빠름 | 적음 | 실시간 처리 |

In [ ]:
# 차원 수를 1024로 줄인 임베딩 객체를 생성합니다.
# 기본 1536차원 대비 약 33% 메모리 절약 효과가 있습니다.
embeddings_1024 = OpenAIEmbeddings(model="text-embedding-3-small", dimensions=1024)

In [ ]:
# 1024차원으로 설정된 임베딩의 실제 벡터 길이를 확인합니다.
# 결과는 1024가 나와야 합니다.
len(embeddings_1024.embed_documents([text])[0])

# 6. 유사도 계산 🎯

임베딩의 **진짜 힘**은 텍스트 간의 **의미적 유사성**을 수치로 측정할 수 있다는 것입니다!

In [ ]:
# 유사도 계산을 위한 다양한 테스트 문장들을 정의합니다.
# 의미적 유사성을 비교하기 위해 한국어와 영어 문장을 혼합했습니다.
sentence1 = "안녕하세요? 반갑습니다."
sentence2 = "안녕하세요? 반갑습니다!"
sentence3 = "안녕하세요? 만나서 반가워요."
sentence4 = "Hi, nice to meet you."
sentence5 = "I like to eat apples."

In [ ]:
# 코사인 유사도 계산을 위해 scikit-learn 라이브러리를 사용합니다.
from sklearn.metrics.pairwise import cosine_similarity

# 모든 테스트 문장들을 리스트로 묶어서 관리합니다.
sentences = [sentence1, sentence2, sentence3, sentence4, sentence5]

# 5개 문장을 모두 1024차원 벡터로 변환합니다 (배치 처리).
embedded_sentences = embeddings_1024.embed_documents(sentences)

In [ ]:
# 두 벡터 간의 코사인 유사도를 계산하는 헬퍼 함수입니다.
# 결과는 -1(완전 반대) ~ 1(완전 동일) 사이의 값으로 나타납니다.
def similarity(a, b):
    return cosine_similarity([a], [b])[0][0]

In [ ]:
# 모든 문장 쌍에 대해 유사도를 계산하고 결과를 출력합니다.
# 중복을 피하기 위해 i < j 조건을 사용합니다.

for i, sentence in enumerate(embedded_sentences):
    for j, other_sentence in enumerate(embedded_sentences):
        if i < j:  # 중복 비교를 피하기 위한 조건
            print(
                f"[유사도 {similarity(sentence, other_sentence):.4f}] {sentences[i]} \t <=====> \t {sentences[j]}"
            )

## 📊 유사도 결과 해석하기

위 결과를 통해 OpenAI Embeddings가 얼마나 **똑똑하게** 문장의 의미를 파악하는지 확인해보세요!

### 🎯 예상되는 결과 패턴

1. **🥇 최고 유사도** (0.99+): `"안녕하세요? 반갑습니다."` ↔ `"안녕하세요? 반갑습니다!"`
   - 거의 동일한 문장 (구두점만 다름)

2. **🥈 높은 유사도** (0.9+): `"안녕하세요? 반갑습니다."` ↔ `"안녕하세요? 만나서 반가워요."`
   - 같은 의미, 다른 표현

3. **🥉 중간 유사도** (0.7-0.9): 한국어 인사말 ↔ `"Hi, nice to meet you."`
   - 같은 의미, 다른 언어 (놀라운 다국어 이해!)

4. **🚫 낮은 유사도** (0.3 이하): 인사말 ↔ `"I like to eat apples."`
   - 완전히 다른 주제

### 💡 실생활 활용 아이디어

- **🔍 검색 엔진**: 정확히 일치하지 않아도 의미가 비슷한 문서 찾기
- **🤖 챗봇**: 다양한 방식으로 표현된 질문을 같은 의도로 인식
- **📚 문서 분류**: 비슷한 주제의 문서들을 자동으로 그룹핑
- **🌐 번역**: 언어는 다르지만 의미가 같은 문장 찾기

---

## 🎉 마무리

축하합니다! 이제 **OpenAI Embeddings**의 핵심 기능들을 모두 경험해보셨습니다:

- ✅ **환경 설정** - API 키와 추적 시스템 구성
- ✅ **모델 선택** - 용도에 맞는 적절한 모델 고르기
- ✅ **쿼리 임베딩** - 단일 텍스트를 벡터로 변환
- ✅ **문서 임베딩** - 여러 문서를 일괄 처리
- ✅ **차원 조정** - 성능과 효율성의 균형 맞추기
- ✅ **유사도 계산** - 텍스트 간 의미적 유사성 측정

### 🚀 다음 단계

이제 배운 내용을 활용해서:
- **RAG (Retrieval-Augmented Generation)** 시스템 구축하기
- **벡터 데이터베이스**와 연동하기  
- **실제 프로젝트**에 임베딩 기능 적용하기

**Happy Coding!** 🎯